In [6]:
import pandas as pd

from google.colab import drive

drive.mount('/content/drive')

Data_Elec = pd.read_excel('/content/drive/My Drive/NUS-ISS AIS Projects/Project 1/Data/Electrical_Parts_V2.xlsx',  engine='openpyxl')
Data_Misc = pd.read_excel('/content/drive/My Drive/NUS-ISS AIS Projects/Project 1/Data/Misc_Parts.xlsx',  engine='openpyxl')

print("Electrical Data :\n")
print(Data_Elec.head())

print("\nMisc Data :\n")
print(Data_Misc.head())


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Electrical Data :

       # DOCTYPE CANCELED DOCCUR  DOCRATE DocStatus  PO NUM Base Type  \
0   3874       I        N    INR      1.0         C   60186        -1   
1   3875       I        N    INR      1.0         C   60186        -1   
2   3954       I        N    INR      1.0         C   60213        -1   
3   8557       I        N    INR      1.0         C   72163        -1   
4  13535       I        N    INR      1.0         C   73639        PR   

   Base Number  Base Line  ... Manually Closed PO PO OWNER Name   PymntGroup  \
0          NaN        NaN  ...                 No           NaN       Net-30   
1          NaN        NaN  ...                 No           NaN       Net-30   
2          NaN        NaN  ...                 No           NaN       Net-30   
3          NaN        NaN  ...                 No           NaN       Net-30   
4     101378.

In [3]:
print(Data_Elec['Part ID'])

0        18165
1        18165
2        18166
3        18167
4        24587
         ...  
11108    36220
11109    18212
11110    18212
11111    18080
11112    18080
Name: Part ID, Length: 11113, dtype: int64


In [9]:
print(Data_Misc['Misc_ID'])

0          MC11560
1          4556800
2                0
3                0
4                0
           ...    
39343    00047513A
39344    00047494A
39345    00047510A
39346      3079000
39347      3079100
Name: Misc_ID, Length: 39348, dtype: object


In [11]:
#Drop the duplicate values from Electrical Data
Data_Elec_Unique = Data_Elec.drop_duplicates(subset=['Part ID'])

#Drop the duplicate values from Misc Data
Data_Misc_Unique = Data_Misc.drop_duplicates(subset=['Misc_ID'])

#Storing in a CSV File
Data_Elec_Unique.to_csv('Electrical_Parts_Unique.csv', index=False)
Data_Misc_Unique.to_csv('Misc_Parts_Unique.csv', index=False)

print('Unique Electrcial Parts: \n', Data_Elec_Unique['Part ID'])
print('\nUnique Miscellaneous Parts: \n', Data_Misc_Unique['Misc_ID'])

Unique Electrcial Parts: 
 0        18165
2        18166
3        18167
4        24587
7        25933
         ...  
11098    29706
11100    35470
11104    18080
11105    18212
11106    36220
Name: Part ID, Length: 3343, dtype: int64

Unique Miscellaneous Parts: 
 0        MC11560
1        4556800
2              0
11       ST00545
12       ST00546
          ...   
39334    3483002
39335    3477601
39336    4248101
39337    3734701
39346    3079000
Name: Misc_ID, Length: 17228, dtype: object


# Replacing the values

Values to be replaced in the column:

ID,
Name,
Supplier,
Supplier ID,
Item Description,
Item Details,
Manufacturer Part Number,
HSN code

In [26]:
# Define a function to find a random replacement part based on similar price and quantity
def find_replacement(mechanical_row, electrical_df, tolerance_price=0.3, tolerance_quantity=0.3):
    price_range = (mechanical_row['PRICE'] * (1 - tolerance_price), mechanical_row['PRICE'] * (1 + tolerance_price))
    quantity_range = (mechanical_row['ORDERED QUANTITY'] * (1 - tolerance_quantity), mechanical_row['ORDERED QUANTITY'] * (1 + tolerance_quantity))

    # Filter electrical parts that fall within the price and quantity range
    matching_parts = electrical_df[
        (electrical_df['PRICE'] >= price_range[0]) & (electrical_df['PRICE'] <= price_range[1]) &
        (electrical_df['ORDERED QUANTITY'] >= quantity_range[0]) & (electrical_df['ORDERED QUANTITY'] <= quantity_range[1])
    ]

    # If there are matching parts, return a random one
    if not matching_parts.empty:
        return matching_parts.sample(1).iloc[0]
    return None

print(Data_Misc['Misc_ID'])

# Iterate over mechanical parts and replace them with electrical parts
Modified_Data_Misc = Data_Misc.copy()

for index, row in Data_Misc.iterrows():
    replacement = find_replacement(row, Data_Elec)
    if replacement is not None:
        Modified_Data_Misc.loc[index, 'Misc_ID'] = replacement['Part ID']
        Modified_Data_Misc.loc[index, 'ITEM NAME'] = replacement['Part Name']
        Modified_Data_Misc.loc[index, 'SUPPLIER NAME'] = replacement['SUPPLIER NAME']
        Modified_Data_Misc.loc[index, 'SUPPLIER CODE'] = replacement['SUPPLIER CODE']
        Modified_Data_Misc.loc[index, 'FG Item Description'] = replacement['FG Item Description']
        Modified_Data_Misc.loc[index, 'Item Details'] = replacement['Item Details']
        Modified_Data_Misc.loc[index, 'UOM'] = replacement['UOM']
        Modified_Data_Misc.loc[index, 'Manufacturer Part No'] = replacement['Manufacturer Part No']
        Modified_Data_Misc.loc[index, 'HSN'] = replacement['HSN']

# Save the updated dataframe with replaced parts
Modified_Data_Misc.to_csv('Modified_Data_Misc.csv', index=False)


0          MC11560
1          4556800
2                0
3                0
4                0
           ...    
39343    00047513A
39344    00047494A
39345    00047510A
39346      3079000
39347      3079100
Name: Misc_ID, Length: 39348, dtype: object


In [33]:
#Define a function to randomly replace null values with the ID having the closest price
def fill_null_misc_id(df, price_column='PRICE'):
 Data_Misc_Copy = Data_Misc.copy()  # Create a copy to avoid modifying the original DataFrame
 null_indices = Data_Misc_Copy['Misc_ID'].isnull()

 for index in Data_Misc_Copy[null_indices].index:
      # Find the row with the closest price
      row_price = Data_Misc_Copy.loc[index, 'PRICE']
      Data_Misc_Copy['price_diff'] = abs(Data_Misc_Copy['PRICE'] - row_price)
      closest_rows = Data_Misc_Copy.loc[~Data_Misc_Copy['Misc_ID'].isnull()].sort_values('price_diff')

      # If there are rows with non-null Misc_ID and close prices
      if not closest_rows.empty:
          # Select a random row from the closest rows
          random_row_index = np.random.choice(closest_rows.index)
          Data_Misc_Copy.loc[index, 'Misc_ID'] = Data_Misc_Copy.loc[random_row_index, 'Misc_ID']
      # If no suitable replacement is found, it will remain null

 Data_Misc_Copy = Data_Misc_Copy.drop('price_diff', axis=1, errors='ignore')
 return Data_Misc_Copy

Data_Misc_Fill = fill_null_misc_id(Data_Misc)

  # Define a function to find a random replacement part based on similar price and quantity
def find_replacement(mechanical_row, electrical_df, tolerance_price=0.3, tolerance_quantity=0.3):
    price_range = (mechanical_row['PRICE'] * (1 - tolerance_price), mechanical_row['PRICE'] * (1 + tolerance_price))
    quantity_range = (mechanical_row['ORDERED QUANTITY'] * (1 - tolerance_quantity), mechanical_row['ORDERED QUANTITY'] * (1 + tolerance_quantity))

    # Filter electrical parts that fall within the price and quantity range
    matching_parts = electrical_df[
        (electrical_df['PRICE'] >= price_range[0]) & (electrical_df['PRICE'] <= price_range[1]) &
        (electrical_df['ORDERED QUANTITY'] >= quantity_range[0]) & (electrical_df['ORDERED QUANTITY'] <= quantity_range[1])
    ]

    # If there are matching parts, return a random one
    if not matching_parts.empty:
        return matching_parts.sample(1).iloc[0]
    return None

print(Data_Misc_Fill['Misc_ID'])

# Iterate over mechanical parts and replace them with electrical parts
Modified_Data_Misc = Data_Misc_Fill.copy()

for index, row in Data_Misc_Fill.iterrows():
    replacement = find_replacement(row, Data_Elec)
    if replacement is not None:
        Modified_Data_Misc.loc[index, 'Misc_ID'] = replacement['Part ID']
        Modified_Data_Misc.loc[index, 'ITEM NAME'] = replacement['Part Name']
        Modified_Data_Misc.loc[index, 'SUPPLIER NAME'] = replacement['SUPPLIER NAME']
        Modified_Data_Misc.loc[index, 'SUPPLIER CODE'] = replacement['SUPPLIER CODE']
        Modified_Data_Misc.loc[index, 'FG Item Description'] = replacement['FG Item Description']
        Modified_Data_Misc.loc[index, 'Item Details'] = replacement['Item Details']
        Modified_Data_Misc.loc[index, 'UOM'] = replacement['UOM']
        Modified_Data_Misc.loc[index, 'Manufacturer Part No'] = replacement['Manufacturer Part No']
        Modified_Data_Misc.loc[index, 'HSN'] = replacement['HSN']

# Save the updated dataframe with replaced parts
Modified_Data_Misc.to_csv('Modified_Data_Misc_without_NULL.csv', index=False)




0          MC11560
1          4556800
2                0
3                0
4                0
           ...    
39343    00047513A
39344    00047494A
39345    00047510A
39346      3079000
39347      3079100
Name: Misc_ID, Length: 39348, dtype: object


In [34]:
# Merge Data_Elec_Unique and Data_Misc_Unique based on 'Part ID' and 'Misc_ID'
Merged_Data = pd.merge(Data_Misc_Unique, Data_Elec_Unique, left_on='Misc_ID', right_on='Part ID', how='left')

# Update columns in Modified_Data_Misc
Modified_Data_Misc = Data_Misc.copy()

for index, row in Merged_Data.iterrows():
  if pd.notna(row['Part ID']):  # Check if 'Part ID' is not NaN
      Modified_Data_Misc.loc[Modified_Data_Misc['Misc_ID'] == row['Misc_ID'], 'SUPPLIER NAME'] = row['SUPPLIER NAME_y']
      Modified_Data_Misc.loc[Modified_Data_Misc['Misc_ID'] == row['Misc_ID'], 'SUPPLIER CODE'] = row['SUPPLIER CODE_y']
      Modified_Data_Misc.loc[Modified_Data_Misc['Misc_ID'] == row['Misc_ID'], 'FG Item Description'] = row['FG Item Description_y']
      Modified_Data_Misc.loc[Modified_Data_Misc['Misc_ID'] == row['Misc_ID'], 'Item Details'] = row['Item Details_y']
      Modified_Data_Misc.loc[Modified_Data_Misc['Misc_ID'] == row['Misc_ID'], 'UOM'] = row['UOM_y']
      Modified_Data_Misc.loc[Modified_Data_Misc['Misc_ID'] == row['Misc_ID'], 'Manufacturer Part No'] = row['Manufacturer Part No_y']
      Modified_Data_Misc.loc[Modified_Data_Misc['Misc_ID'] == row['Misc_ID'], 'HSN'] = row['HSN_y']

# Save the updated dataframe
Modified_Data_Misc.to_csv('Modified_Data_Misc_Merged.csv', index=False)

In [ ]:
#Print Number of Matching ID from two data sets

